# Predict Severity of Car Accident

## Business Problem
Traffic accident has been a more and more importante issue threatening people’s public safety. Therefore, vehicle collision prediction based on relevant data and promptly reminding people may help to reduce accidents.

In this project, we will try to predict the severity of accidents given the data that could affact it, like weather, road condition, car speeding, etc. in the city **Seatle, USA.**

The stakeholder of this project can be drivers who can use the prediction to get more careful o the traffic emergency  department to get more prepared for the possible accident.

## Data
Based on definition of our problem, we could include the following factors that influence the severity of a traffic accident:
- road condition (ROADCOND)
- weather condition (WEATHER)
- junction (JUNCTIONTYPE)
- light conditions (LIGHTCOND)
- location
- number of people involved
- number of vehicles involved in
- car speeding (SPEEDING) 

Given the data provided, I'll use the following fields: 
- road condition (ROADCOND)
- weather condition (WEATHER)
- junction (JUNCTIONTYPE)
- light conditions (LIGHTCOND)
- location

The number of people involved and number of vehicles involved in won't be taken into account as we may not know these data when we want to predict an accident.

The car speeding (SPEEDING) field is not used because in this dataset, it means Whether or not speeding was a factor in the collision. (Y/N) and all of them is Yes which will not contribute to this project.